In [3]:

import torch
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
from tournamentgym import TournamentEnv
import os
import random
from dataclasses import dataclass
import torch
from torch.utils.data import Dataset
from transformers import DecisionTransformerConfig, DecisionTransformerModel, Trainer, TrainingArguments
torch.cuda.is_available()
from tqdm .autonotebook import tqdm
from imitation.algorithms import sqil,bc
from stable_baselines3.common.evaluation import evaluate_policy


In [4]:
from imitation.data.rollout import TrajectoryAccumulator
from imitation.data.huggingface_utils import trajectories_to_dataset
# from imitation.data import serialize
from imitation.data.rollout import rollout_stats
from imitation.data import serialize
from imitation.algorithms import sqil

In [5]:
MW='M'

team_data=pd.read_csv(f'Process_data/{MW}_pm_w_names.csv')
env=TournamentEnv(team_stats=team_data,
                     season=2022,
                     verbose=False,
                     discrete=False,
                     shuffle=True,
                     loading_bar=False,
                     exclude_seasons=[2023])
torch.cuda.is_available()

True

In [6]:
env.cheat_action() 

array([1., 0.])

In [7]:
num_dems=1000
traj_acum=TrajectoryAccumulator()
demonstrations=[]
for i in tqdm(range(num_dems)):
    obs,info=env.reset()
    terminated=False
    step={'obs':obs}
    traj_acum.add_step(key=i,step_dict=step)
    j=0
    while not terminated:
        act=env.cheat_action() 
        j+=1
        if i%10!=0:
            if (j%10)==0:
                act=env.action_space.sample()
                
        next_obs, reward, terminated,truncated, info=env.step(action=act,)
        
        step={'obs':torch.from_numpy(np.array(obs)).cpu(),
            'acts':torch.from_numpy(act).cpu(),
            'rews':torch.from_numpy(np.array(reward)).cpu(),
            'infos':info}
        traj_acum.add_step(key=i,
                        step_dict=step,
                        )
        # print(reward,train_env.trade_week_start,train_env.current_time)
        obs=next_obs
    
    
    traj=traj_acum.finish_trajectory(key=i,terminal=terminated)
    demonstrations.append(traj)

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
# stats = rollout_stats(demonstrations)
# print(stats["return_mean"])

In [9]:
torch.cuda.is_available()
torch.device('cuda')

device(type='cuda')

In [10]:
from imitation.util.logger import HierarchicalLogger,make_output_format,configure

In [11]:
format=make_output_format(_format='tensorboard', log_dir='decision_models/', log_suffix='', max_length=50)

In [10]:
hlog=HierarchicalLogger(default_logger=configure('copy_models/', ('log','csv'))
                    
                                                     )
hlog.output_formats.append(format)


In [11]:
    
bc_trainer = bc.BC(
                    observation_space=env.observation_space,
                    action_space=env.action_space,
                    demonstrations=demonstrations,
                    rng = np.random.default_rng(1),
                    device=torch.device('cpu'),
                    # custom_logger=configure(folder='decision_models/', format_strs='tensorboard')
                    custom_logger=hlog
                )
# bc_trainer.policy
bc_trainer.train(
    n_epochs=1000,
                #  n_batches=10000,
                 log_interval=10000,
                 progress_bar =True)

1147253batch [1:07:48, 275.52batch/s]

In [1]:
reward, _ = evaluate_policy(bc_trainer.policy, env, 10)
print("Reward:", reward)

NameError: name 'evaluate_policy' is not defined

In [2]:
env=TournamentEnv(team_stats=team_data,
                     season=2022,
                     verbose=False,
                     discrete=False,
                     loading_bar=False,
                     shuffle=True
                     )

NameError: name 'team_data' is not defined

In [ ]:
obs,info=env.reset(season=2023)
terminated=False
while not terminated:
    action=bc_trainer.policy.predict(obs,deterministic=True)[0]
    next_obs, reward, terminated,truncated, info=env.step(action=action)

    obs=next_obs
info

In [ ]:
serialize.save('demonstrations/Mens_demonstrations.pkl', demonstrations)
